In [1]:
from interface import *

Matplotlib is building the font cache; this may take a moment.


ImportError: cannot import name 'vtkCapsuleSource' from 'vtkmodules.vtkFiltersSources' (C:\Users\amen0052\AppData\Local\anaconda3\envs\geoclassifmps\Lib\site-packages\vtkmodules\vtkFiltersSources.cp39-win_amd64.pyd)

In [ ]:
verbose=True
arg_seed=123
arg_n_ti=1
arg_ti_pct_area=90
arg_num_shape=15
arg_aux_vars=["grid_lmp","grid_grv","grid_mag"]
arg_output_dir="./output/tmp"

In [ ]:
params, \
shorten, \
nvar, sim_var, auxTI_var, auxSG_var, condIm_var, names_var, types_var, outputVarFlag, \
nr, nc = get_simulation_info(arg_seed, arg_n_ti, arg_ti_pct_area, arg_num_shape, arg_aux_vars, arg_output_dir)